In [1]:
from transformers import AutoProcessor, Wav2Vec2Model
import torch
from datasets import load_dataset
from collect_data import *

/home/matteoc/miniconda3/envs/speech-meg/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
dataset = load_dataset("hf-internal-testing/librispeech_asr_demo", "clean", split="validation")
dataset = dataset.sort("id")
sampling_rate = dataset.features["audio"].sampling_rate

processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

inputs = processor(dataset[0]["audio"]["array"], sampling_rate=sampling_rate, return_tensors="pt")



Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
inputs.input_values.shape

torch.Size([1, 93680])

In [7]:
with torch.no_grad():
    outputs = model(**inputs)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

[1, 292, 768]

## Dati Nostri

In [2]:
stimuli_path = meg_path + '/stimuli/audio'
wav_files_duration = {}

for filename in os.listdir(stimuli_path):
    if filename.endswith('.wav'): 
        file_path = os.path.join(stimuli_path, filename)
        y, sr = librosa.load(file_path, sr=None)
        duration = librosa.get_duration(y=y, sr=sr)
        wav_files_duration[filename.rstrip('.wav')] = duration
print('WAVE FILES DURATION: ',wav_files_duration)
print('WAVE FILES WITH\ NUMBERS: ',task)
wav_list_without_numb = list(task.keys())


WAVE FILES DURATION:  {'lw1': 318.1481632653061, 'lw1_3': 53.072426303854876, 'lw1_2': 93.7512925170068, 'lw1_1': 75.44276643990929, 'lw1_0': 95.88167800453515, 'easy_money_7': 129.30312925170068, 'easy_money_6': 123.42730158730159, 'easy_money_4': 163.68920634920636, 'easy_money_5': 142.36866213151927, 'easy_money_3': 141.97519274376418, 'easy_money_0': 143.12875283446712, 'easy_money_2': 180.43360544217688, 'easy_money_1': 104.10834467120182, 'cable_spool_fort_4': 123.26390022675737, 'cable_spool_fort_5': 54.69605442176871, 'cable_spool_fort_3': 105.82820861678005, 'cable_spool_fort_2': 137.65614512471655, 'cable_spool_fort_1': 134.8726984126984, 'cable_spool_fort_0': 100.59777777777778, 'the_black_willow_9': 188.23668934240362, 'the_black_willow_8': 142.4142403628118, 'the_black_willow_7': 172.399410430839, 'the_black_willow_6': 128.21628117913832, 'the_black_willow_5': 81.94331065759637, 'the_black_willow_4': 110.45659863945578, 'the_black_willow_2': 125.23859410430839, 'the_black_

In [9]:
audio_input = []

In [19]:
audio_name = wav_list_without_numb[3]
subject = patient[0]
print('PATIENT: ', subject)
print('AUDIO_NAME: ', audio_name)


PATIENT:  01
AUDIO_NAME:  the_black_willow


In [20]:
for i in range(len(session)):
    print("SESSION: ", session[i])
    story_uid = int(task[audio_name])
    print("STORY_UID_OR_TASK: ", story_uid)
    raw = get_bids_raw(meg_path, subject, session[i], str(story_uid))
    for z in range(len(the_black_willow)):
        print("SOUND_ID: ", float(the_black_willow[z]))
        epochs_data = get_epochs(raw, float(story_uid), float(the_black_willow[z]))
        if (i == 0 and subject == '01'):
            audio_path = stimuli_path + '/' + audio_name + '_' + str(z) + '.wav'
            data_audio_chunks = []
            epoch_spectr = get_meg_from_raw_epochs(epochs_data)
            for j in range(epoch_spectr.shape[0]):
                start = epochs_data[j]._metadata["start"].item()
                duration = 3
                y, sr = librosa.load(audio_path, sr=sampling_audio, offset=start, duration=duration)
                if (y.shape[0] < duration*sampling_audio):   
                    # make padding         
                    pad_width = duration*sampling_audio - y.shape[0]
                    y = np.pad(y, (0, pad_width), mode='constant', constant_values=0)
                data_audio_chunks.append(y)
            audio_tensor_chunk = torch.tensor(data_audio_chunks)
            audio_input.append(audio_tensor_chunk)
            print('AUDIO_SPECTR_SHAPE: ', audio_tensor_chunk.shape)

SESSION:  0
STORY_UID_OR_TASK:  3
Reading 0 ... 1868999  =      0.000 ...  1868.999 secs...


/home/matteoc/spect-to-meg/code/collect_data.py:47: RuntimeWarning: The unit for channel(s) MISC 001, MISC 002, MISC 003, MISC 004, MISC 005, MISC 006, MISC 007, MISC 008, MISC 009, MISC 010, MISC 011, MISC 012, MISC 013, MISC 014, MISC 015, MISC 016, MISC 017, MISC 018, MISC 019, MISC 020, MISC 021, MISC 022, MISC 023, MISC 024, MISC 025, MISC 026, MISC 027, MISC 028, MISC 029, MISC 030, MISC 031, MISC 032 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6601 samples (6.601 s)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
SOUND_ID:  0.0
Adding metadata with 18 columns
358 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 358 events and 3201 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)
AUDIO_SPECTR_SHAPE:  torch.Size([358, 48000])
SOUND_ID:  1.0
Adding metada

/home/matteoc/spect-to-meg/code/collect_data.py:47: RuntimeWarning: The unit for channel(s) MISC 001, MISC 002, MISC 003, MISC 004, MISC 005, MISC 006, MISC 007, MISC 008, MISC 009, MISC 010, MISC 011, MISC 012, MISC 013, MISC 014, MISC 015, MISC 016, MISC 017, MISC 018, MISC 019, MISC 020, MISC 021, MISC 022, MISC 023, MISC 024, MISC 025, MISC 026, MISC 027, MISC 028, MISC 029, MISC 030, MISC 031, MISC 032 has changed from V to NA.
  raw = mne_bids.read_raw_bids(bids_path, verbose=False)


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 0.5 - 30 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 0.50
- Lower transition bandwidth: 0.50 Hz (-6 dB cutoff frequency: 0.25 Hz)
- Upper passband edge: 30.00 Hz
- Upper transition bandwidth: 7.50 Hz (-6 dB cutoff frequency: 33.75 Hz)
- Filter length: 6601 samples (6.601 s)

NOTE: pick_types() is a legacy function. New code should use inst.pick(...).
SOUND_ID:  0.0
Adding metadata with 18 columns
358 matching events found
Applying baseline correction (mode: mean)
0 projection items activated
Using data from preloaded Raw for 358 events and 3201 original time points ...
0 bad epochs dropped
Applying baseline correction (mode: mean)
SOUND_ID:  1.0
Adding metadata with 18 columns
262 matching events found
A

In [27]:
print(len(audio_input))
combined_tensor = torch.cat(audio_input, dim=0)

30


In [28]:
# save_pred_target = os.path.join(meg_path, 'collect_data/wave2vec_input.pt')
# torch.save(combined_tensor, save_pred_target)

w2v_input_tensor = torch.load(os.path.join(meg_path, 'collect_data/wave2vec_input.pt'))
print(w2v_input_tensor.shape)


torch.Size([8561, 48000])


In [10]:
processor = AutoProcessor.from_pretrained("facebook/wav2vec2-base-960h")
model = Wav2Vec2Model.from_pretrained("facebook/wav2vec2-base-960h")

inputs = processor(audio_input[0], sampling_rate=sampling_audio, return_tensors="pt")

Some weights of Wav2Vec2Model were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
inputs_input_values = inputs.input_values.squeeze(0)

In [19]:
with torch.no_grad():
    outputs = model(inputs_input_values)

last_hidden_states = outputs.last_hidden_state
list(last_hidden_states.shape)

[180, 149, 768]